# Red de Autopistas - Buenos Aires

Este notebook modela el problema de construcción de una red de autopistas que conecte ciudades importantes de Buenos Aires, minimizando la longitud total de infraestructura a construir.

## Comparación de Algoritmos:
1. **MST (Minimum Spanning Tree)** - Conecta directamente solo las ciudades principales
2. **Dreyfus-Wagner** - Permite construir autopistas pasando por ciudades intermedias para reducir la longitud total

## Escenario:
- **Ciudades Principales (Terminales)**: Ciudades importantes que DEBEN estar conectadas por autopista
- **Ciudades Intermedias (Steiner)**: Ciudades por donde puede pasar la autopista si esto reduce la longitud total
- **Objetivo**: Minimizar los kilómetros totales de autopista a construir conectando todas las ciudades principales

**Ventaja del enfoque Steiner**: Si construir autopistas pasando por ciudades intermedias reduce la distancia total, es más económico (menos infraestructura).

In [1]:
# Importar librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations, chain
import time
import folium
from folium import plugins
import requests
import json

print("Librerías importadas correctamente!")

Librerías importadas correctamente!


## Definición de las Ciudades de Buenos Aires

Coordenadas reales (latitud, longitud) de ciudades de la provincia de Buenos Aires que se consideran para el proyecto de autopistas:

In [2]:
# Coordenadas reales (latitud, longitud) de ciudades de Buenos Aires
ciudades = {
    # CIUDADES PRINCIPALES - Deben estar conectadas por autopista (obligatorio)
    'La Plata': (-34.9205, -57.9536),         # Capital provincial
    'Mar del Plata': (-38.0055, -57.5426),    # Puerto importante
    'Bahía Blanca': (-38.7183, -62.2663),     # Puerto sur
    'Tandil': (-37.3217, -59.1332),           # Centro serrano
    'Pergamino': (-33.8897, -60.5736),        # Norte productivo
    'General Villegas': (-35.0331, -63.0122), # Zona oeste
    'San Carlos de Bolívar': (-36.2333, -61.1167),  # Centro-oeste
    
    # CIUDADES INTERMEDIAS - Pueden ser atravesadas por la autopista si conviene
    'Junín': (-34.5853, -60.9445),            # Zona oeste
    'Dolores': (-36.3131, -57.6792),          # Entre La Plata y Mar del Plata
    'Azul': (-36.7770, -59.8586),             # Centro de la provincia
    'Olavarría': (-36.8927, -60.3225),        # Zona central
    'Tres Arroyos': (-38.3740, -60.2789),     # Sur
    'Balcarce': (-37.8462, -58.2553),         # Cerca de Mar del Plata
    'Chivilcoy': (-34.8967, -60.0189),        # Camino a Junín
    'Mercedes': (-34.6515, -59.4307),         # Zona noroeste
    'Pehuajó': (-35.8120, -61.8982),          # Zona central oeste
    'Trenque Lauquen': (-35.9742, -62.7303),  # Oeste
    'Necochea': (-38.5545, -58.7408),         # Costa atlántica
    'Rauch': (-36.7725, -59.0931),            # Centro
    'Las Flores': (-36.0236, -59.0964),       # Centro norte
    'Saladillo': (-35.6383, -59.7786),        # Centro
    'Bragado': (-35.1189, -60.4867),          # Noroeste
    'Lobos': (-35.1858, -59.0964),            # Norte
    'Chascomús': (-35.5744, -58.0119),        # Norte
    'General Madariaga': (-37.0058, -57.1314),# Costa
    'Guaminí': (-37.0167, -62.4333),          # Sudoeste
    '9 de Julio': (-35.4439, -60.8839),       # Centro
    'Coronel Pringles': (-37.9833, -61.3500), # Sur
    'Coronel Suárez': (-37.4500, -61.9333),   # Sudoeste
    'General Belgrano': (-35.7667, -58.4833), # Norte
}

# Clasificación de ciudades
terminales = ['La Plata', 'Mar del Plata', 'Bahía Blanca', 'Tandil', 'Pergamino', 'San Carlos de Bolívar']
steiner_nodes = [ciudad for ciudad in ciudades.keys() if ciudad not in terminales]

print(f"Ciudades Principales ({len(terminales)}): {terminales}")
print(f"\nCiudades Intermedias ({len(steiner_nodes)}): {steiner_nodes}")
print(f"\nTotal de ciudades: {len(ciudades)}")

Ciudades Principales (6): ['La Plata', 'Mar del Plata', 'Bahía Blanca', 'Tandil', 'Pergamino', 'San Carlos de Bolívar']

Ciudades Intermedias (24): ['General Villegas', 'Junín', 'Dolores', 'Azul', 'Olavarría', 'Tres Arroyos', 'Balcarce', 'Chivilcoy', 'Mercedes', 'Pehuajó', 'Trenque Lauquen', 'Necochea', 'Rauch', 'Las Flores', 'Saladillo', 'Bragado', 'Lobos', 'Chascomús', 'General Madariaga', 'Guaminí', '9 de Julio', 'Coronel Pringles', 'Coronel Suárez', 'General Belgrano']

Total de ciudades: 30


## Crear Grafo con Distancias Reales

Calculamos distancias usando la fórmula de Haversine para coordenadas geográficas:

In [3]:
def haversine_distance(coord1, coord2):
    """
    Calcula la distancia entre dos puntos geográficos usando la fórmula de Haversine.
    
    Parameters:
    - coord1, coord2: Tuplas (latitud, longitud) en grados
    
    Returns: Distancia en kilómetros
    """
    lat1, lon1 = np.radians(coord1)
    lat2, lon2 = np.radians(coord2)
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Radio de la Tierra en km
    R = 6371
    return R * c


def crear_grafo_completo(ciudades_dict):
    """
    Crea un grafo completo con todas las ciudades conectadas.
    Las aristas tienen peso = distancia real en km.
    """
    G = nx.Graph()
    
    # Agregar nodos con posiciones
    for ciudad, coords in ciudades_dict.items():
        G.add_node(ciudad, pos=coords)
    
    # Agregar aristas con distancias
    ciudades_lista = list(ciudades_dict.keys())
    for i, ciudad1 in enumerate(ciudades_lista):
        for ciudad2 in ciudades_lista[i+1:]:
            dist = haversine_distance(ciudades_dict[ciudad1], ciudades_dict[ciudad2])
            G.add_edge(ciudad1, ciudad2, weight=dist)
    
    return G


# Crear el grafo completo
G = crear_grafo_completo(ciudades)
print(f"Grafo creado: {G.number_of_nodes()} nodos, {G.number_of_edges()} aristas")
print(f"\nEjemplo de distancias:")
print(f"La Plata - Mar del Plata: {G['La Plata']['Mar del Plata']['weight']:.1f} km")
print(f"La Plata - Bahía Blanca: {G['La Plata']['Bahía Blanca']['weight']:.1f} km")
print(f"Junín - Pergamino: {G['Junín']['Pergamino']['weight']:.1f} km")

Grafo creado: 30 nodos, 435 aristas

Ejemplo de distancias:
La Plata - Mar del Plata: 345.0 km
La Plata - Bahía Blanca: 570.6 km
Junín - Pergamino: 84.5 km


## Mapa General de Todas las Ciudades

Visualización de todas las ciudades consideradas para el proyecto de autopistas.

In [4]:
def visualizar_grafo_completo_mapa(G, terminales, titulo="Ciudades del Proyecto de Autopistas"):
    """
    Visualiza todas las ciudades sobre un mapa geográfico real de Buenos Aires.
    Solo muestra las ciudades sin las conexiones de autopista.
    """
    # Centro del mapa
    centro_lat = -36.5
    centro_lon = -59.5
    
    # Crear mapa base
    mapa = folium.Map(
        location=[centro_lat, centro_lon],
        zoom_start=7,
        tiles='OpenStreetMap'
    )
    
    # Agregar título y leyenda
    title_html = f'''
        <div style="position: fixed; 
                    top: 10px; left: 50px; width: 450px; height: 90px; 
                    background-color: white; border:2px solid grey; z-index:9999; 
                    font-size:14px; font-weight: bold; padding: 10px">
            {titulo}<br>
            <div style="margin-top: 10px; font-size: 12px;">
                <span style="color:red;">🔴 Rojos:</span> Ciudades Principales (deben conectarse)<br>
                <span style="color:lightblue;">🔵 Celestes:</span> Ciudades Intermedias (pueden usarse si conviene)
            </div>
        </div>
    '''
    mapa.get_root().html.add_child(folium.Element(title_html))
    
    # Dibujar nodos (sin aristas)
    for nodo in G.nodes():
        coord = ciudades[nodo]
        
        # Determinar estilo según tipo de nodo
        if nodo in terminales:
            color = 'red'
            icon = 'star'
            tamaño = 12
            descripcion = 'Ciudad Principal (debe tener acceso a autopista)'
        else:
            color = 'lightblue'
            icon = 'circle'
            tamaño = 8
            descripcion = 'Ciudad Intermedia (puede ser atravesada por autopista)'
        
        # Agregar marcador
        folium.Marker(
            location=[coord[0], coord[1]],
            popup=f"<b>{nodo}</b><br>{descripcion}",
            tooltip=nodo,
            icon=folium.Icon(color=color, icon=icon, prefix='fa', icon_color='white')
        ).add_to(mapa)
        
        # Agregar círculo alrededor
        folium.CircleMarker(
            location=[coord[0], coord[1]],
            radius=tamaño,
            color=color,
            fill=True,
            fillColor=color,
            fillOpacity=0.4,
            weight=2
        ).add_to(mapa)
    
    return mapa

# Crear y mostrar el mapa
mapa_completo = visualizar_grafo_completo_mapa(G, terminales)
mapa_completo

## Implementación del Algoritmo MST

Conecta solo las ciudades principales directamente usando el árbol de expansión mínima:

In [5]:
def calcular_mst(G, terminales):
    """
    Calcula el MST conectando solo las ciudades principales.
    Construye autopistas directamente entre las ciudades principales usando las rutas más cortas.
    """
    # Crear subgrafo con solo ciudades principales
    G_terminales = nx.Graph()
    
    # Agregar aristas entre ciudades principales con peso = camino más corto
    for i, t1 in enumerate(terminales):
        for t2 in terminales[i+1:]:
            # Distancia del camino más corto
            try:
                dist = nx.shortest_path_length(G, t1, t2, weight='weight')
                G_terminales.add_edge(t1, t2, weight=dist)
            except nx.NetworkXNoPath:
                continue
    
    # Calcular MST
    mst = nx.minimum_spanning_tree(G_terminales, weight='weight')
    
    # Expandir a caminos completos en el grafo original
    G_mst_expandido = nx.Graph()
    for u, v in mst.edges():
        camino = nx.shortest_path(G, u, v, weight='weight')
        for i in range(len(camino)-1):
            peso = G[camino[i]][camino[i+1]]['weight']
            G_mst_expandido.add_edge(camino[i], camino[i+1], weight=peso)
    
    return G_mst_expandido


# Calcular MST
start_time = time.time()
mst_solution = calcular_mst(G, terminales)
mst_time = time.time() - start_time

mst_cost = sum(data['weight'] for _, _, data in mst_solution.edges(data=True))
print(f"MST calculado en {mst_time:.4f} segundos")
print(f"Longitud total de autopista (MST): {mst_cost:.2f} km")
print(f"Ciudades en la red: {mst_solution.number_of_nodes()}")
print(f"Tramos de autopista: {mst_solution.number_of_edges()}")

MST calculado en 0.0040 segundos
Longitud total de autopista (MST): 1199.32 km
Ciudades en la red: 6
Tramos de autopista: 5


## Implementación del Algoritmo Dreyfus-Wagner

Algoritmo exacto que permite construir autopistas pasando por ciudades intermedias para minimizar la longitud total de infraestructura:

In [6]:
def dreyfus_wagner(G, terminals):
    """
    Implementación CORRECTA del algoritmo Dreyfus-Wagner para Steiner Tree.
    
    Encuentra la solución ÓPTIMA usando programación dinámica.
    Permite construir autopistas pasando por ciudades intermedias para minimizar la longitud total.
    
    Parameters:
    - G: Grafo completo con pesos en las aristas
    - terminals: Lista de ciudades principales que deben estar conectadas
    
    Returns: (Steiner tree, longitud total)
    
    Complejidad: O(3^n * n^2) donde n = |terminals|
    """
    if len(terminals) < 2:
        raise ValueError("Se necesitan al menos 2 ciudades principales")
    
    n = G.number_of_nodes()
    terminals_set = set(terminals)
    k = len(terminals)
    
    # Precalcular shortest paths entre todos los pares de nodos (distancias y caminos)
    dist = {}
    paths = {}
    for source in G.nodes():
        dist[source], paths[source] = nx.single_source_dijkstra(G, source, weight='weight')
    
    # dp[v][S] = longitud mínima del árbol Steiner que conecta el subconjunto S ⊆ terminals
    #            y está enraizado en el nodo v
    dp = [{} for _ in range(n)]
    parent = [{} for _ in range(n)]  # Para reconstruir el árbol
    
    # Crear mapeo de nodos a índices
    node_list = list(G.nodes())
    node_to_idx = {node: i for i, node in enumerate(node_list)}
    
    # Mapeo de terminales a índices para usar en bitmasks
    term_list = sorted(terminals)
    term_to_idx = {t: i for i, t in enumerate(term_list)}
    
    # Inicialización: subconjuntos de tamaño 1
    for v_idx, v in enumerate(node_list):
        for t in terminals:
            subset = 1 << term_to_idx[t]  # Bitmask con solo el terminal t
            if v == t:
                dp[v_idx][subset] = 0
                parent[v_idx][subset] = ('terminal', t)
            else:
                dp[v_idx][subset] = dist[v][t]
                parent[v_idx][subset] = ('path', t)
    
    # Iterar sobre todos los subconjuntos de terminales (de tamaño 2 a k)
    for subset_size in range(2, k + 1):
        # Generar todos los subconjuntos de tamaño subset_size
        from itertools import combinations
        for subset_terminals in combinations(term_list, subset_size):
            # Convertir a bitmask
            subset = 0
            for t in subset_terminals:
                subset |= (1 << term_to_idx[t])
            
            # FASE 1: Para cada nodo v, calcular ramificaciones (combinaciones de subárboles)
            for v_idx, v in enumerate(node_list):
                min_cost = float('inf')
                best_partition = None
                
                # Iterar sobre todas las particiones no triviales de subset
                subset1 = subset
                while subset1 > 0:
                    subset1 = (subset1 - 1) & subset
                    if subset1 == 0:
                        break
                    
                    subset2 = subset ^ subset1
                    
                    if subset1 in dp[v_idx] and subset2 in dp[v_idx]:
                        cost = dp[v_idx][subset1] + dp[v_idx][subset2]
                        if cost < min_cost:
                            min_cost = cost
                            best_partition = (subset1, subset2)
                
                dp[v_idx][subset] = min_cost
                if best_partition:
                    parent[v_idx][subset] = ('branch', best_partition[0], best_partition[1])
            
            # FASE 2: Expandir soluciones entre diferentes nodos usando shortest paths
            # Este paso debe estar DESPUÉS de calcular todas las ramificaciones
            for v_idx, v in enumerate(node_list):
                for u_idx, u in enumerate(node_list):
                    if u != v and subset in dp[u_idx]:
                        cost = dp[u_idx][subset] + dist[u][v]
                        if cost < dp[v_idx][subset]:
                            dp[v_idx][subset] = cost
                            parent[v_idx][subset] = ('extend', u, subset)
    
    # Encontrar el nodo raíz óptimo para el conjunto completo de terminales
    full_subset = (1 << k) - 1  # Todos los terminales
    min_cost = float('inf')
    best_root_idx = None
    
    for v_idx in range(n):
        if full_subset in dp[v_idx] and dp[v_idx][full_subset] < min_cost:
            min_cost = dp[v_idx][full_subset]
            best_root_idx = v_idx
    
    best_root = node_list[best_root_idx]
    
    # Reconstruir el árbol usando los caminos calculados
    steiner_tree = nx.Graph()
    for node in G.nodes():
        if 'pos' in G.nodes[node]:
            steiner_tree.add_node(node, pos=G.nodes[node]['pos'])
        else:
            steiner_tree.add_node(node)
    
    def add_path_edges(u, v):
        """Agregar todas las aristas del camino más corto entre u y v"""
        if u == v:
            return
        path = paths[u][v]
        for i in range(len(path) - 1):
            n1, n2 = path[i], path[i+1]
            if not steiner_tree.has_edge(n1, n2):
                weight = G[n1][n2]['weight']
                steiner_tree.add_edge(n1, n2, weight=weight)
    
    def reconstruct(v, subset):
        """Reconstruir árbol recursivamente desde la tabla DP"""
        v_idx = node_to_idx[v]
        
        if subset not in parent[v_idx]:
            return
        
        p = parent[v_idx][subset]
        
        if p[0] == 'terminal':
            # Caso base: v es un terminal
            return
        elif p[0] == 'path':
            # Caso base: conectar v a un terminal t
            t = p[1]
            add_path_edges(v, t)
        elif p[0] == 'branch':
            # v es un nodo de ramificación
            subset1, subset2 = p[1], p[2]
            reconstruct(v, subset1)
            reconstruct(v, subset2)
        elif p[0] == 'extend':
            # Extender desde otro nodo u
            u, sub = p[1], p[2]
            add_path_edges(u, v)
            reconstruct(u, sub)
    
    reconstruct(best_root, full_subset)
    
    # Limpiar nodos no usados
    used_nodes = set()
    for u, v in steiner_tree.edges():
        used_nodes.add(u)
        used_nodes.add(v)
    
    unused = [n for n in steiner_tree.nodes() if n not in used_nodes]
    steiner_tree.remove_nodes_from(unused)
    
    # Calcular peso real del árbol
    actual_weight = sum(steiner_tree[u][v]['weight'] for u, v in steiner_tree.edges())
    
    return steiner_tree, actual_weight


# Calcular solución Dreyfus-Wagner
print("Calculando solución Dreyfus-Wagner...")
start_time = time.time()
dw_solution, dw_cost = dreyfus_wagner(G, terminales)
dw_time = time.time() - start_time

print(f"\nDreyfus-Wagner calculado en {dw_time:.4f} segundos")
print(f"Longitud total de autopista (Dreyfus-Wagner): {dw_cost:.2f} km")
print(f"Ciudades en la red: {dw_solution.number_of_nodes()}")
print(f"Tramos de autopista: {dw_solution.number_of_edges()}")

Calculando solución Dreyfus-Wagner...

Dreyfus-Wagner calculado en 0.0131 segundos
Longitud total de autopista (Dreyfus-Wagner): 1132.43 km
Ciudades en la red: 8
Tramos de autopista: 7


## Comparación de Resultados

In [7]:
# Comparación numérica
mejora = ((mst_cost - dw_cost) / mst_cost) * 100

print("=" * 70)
print("COMPARACIÓN DE ALGORITMOS - RED DE AUTOPISTAS BUENOS AIRES")
print("=" * 70)
print(f"\nCiudades principales a conectar: {len(terminales)}")
print(f"Ciudades intermedias disponibles: {len(steiner_nodes)}")
print(f"\n{'Algoritmo':<20} {'Long. Total (km)':<15} {'Ciudades':<10} {'Tramos':<10} {'Tiempo (s)'}")
print("-" * 70)
print(f"{'MST':<20} {mst_cost:<15.2f} {mst_solution.number_of_nodes():<10} {mst_solution.number_of_edges():<10} {mst_time:.4f}")
print(f"{'Dreyfus-Wagner':<20} {dw_cost:<15.2f} {dw_solution.number_of_nodes():<10} {dw_solution.number_of_edges():<10} {dw_time:.4f}")
print("=" * 70)
print(f"\nMejora de Dreyfus-Wagner sobre MST: {mejora:.2f}%")
print(f"Ahorro en infraestructura: {mst_cost - dw_cost:.2f} km de autopista menos")

# Ciudades intermedias utilizadas
steiner_usados = [n for n in dw_solution.nodes() if n in steiner_nodes]
print(f"\nCiudades intermedias utilizadas por Dreyfus-Wagner: {len(steiner_usados)}")
if steiner_usados:
    print(f"La autopista pasa por: {', '.join(steiner_usados)}")

COMPARACIÓN DE ALGORITMOS - RED DE AUTOPISTAS BUENOS AIRES

Ciudades principales a conectar: 6
Ciudades intermedias disponibles: 24

Algoritmo            Long. Total (km) Ciudades   Tramos     Tiempo (s)
----------------------------------------------------------------------
MST                  1199.32         6          5          0.0040
Dreyfus-Wagner       1132.43         8          7          0.0131

Mejora de Dreyfus-Wagner sobre MST: 5.58%
Ahorro en infraestructura: 66.89 km de autopista menos

Ciudades intermedias utilizadas por Dreyfus-Wagner: 2
La autopista pasa por: Olavarría, Chivilcoy


## Análisis de Tramos de Autopista

Veamos los tramos específicos de cada solución:

In [8]:
def analizar_rutas(solucion, nombre_algoritmo):
    """
    Muestra los tramos de autopista de la solución.
    """
    print(f"\n{'='*60}")
    print(f"TRAMOS DE AUTOPISTA - {nombre_algoritmo}")
    print(f"{'='*60}\n")
    
    rutas = sorted(solucion.edges(data=True), 
                   key=lambda x: x[2]['weight'], reverse=True)
    
    print(f"{'Ciudad A':<20} {'Ciudad B':<20} {'Longitud (km)'}")
    print("-" * 60)
    
    for origen, destino, data in rutas:
        print(f"{origen:<20} {destino:<20} {data['weight']:>10.1f}")
    
    print("-" * 60)
    print(f"{'LONGITUD TOTAL:':<40} {sum(d['weight'] for _, _, d in rutas):>10.1f}")

analizar_rutas(mst_solution, "MST")
analizar_rutas(dw_solution, "DREYFUS-WAGNER")


TRAMOS DE AUTOPISTA - MST

Ciudad A             Ciudad B             Longitud (km)
------------------------------------------------------------
Bahía Blanca         San Carlos de Bolívar      294.3
La Plata             Pergamino                 266.3
Pergamino            San Carlos de Bolívar      265.2
Tandil               San Carlos de Bolívar      214.1
Mar del Plata        Tandil                    159.3
------------------------------------------------------------
LONGITUD TOTAL:                              1199.3

TRAMOS DE AUTOPISTA - DREYFUS-WAGNER

Ciudad A             Ciudad B             Longitud (km)
------------------------------------------------------------
Bahía Blanca         Olavarría                 265.3
La Plata             Chivilcoy                 188.3
San Carlos de Bolívar Chivilcoy                 178.7
Mar del Plata        Tandil                    159.3
Pergamino            Chivilcoy                 123.0
Tandil               Olavarría                 115.8

In [9]:
def obtener_ruta_real(coord_origen, coord_destino):
    """
    Obtiene la ruta real entre dos coordenadas usando OSRM (routing de carreteras reales).
    Retorna las coordenadas de la ruta para dibujar en el mapa.
    """
    try:
        # Usar OSRM (Open Source Routing Machine) - servicio público
        # Formato: lon,lat (invertido!)
        url = f"http://router.project-osrm.org/route/v1/driving/{coord_origen[1]},{coord_origen[0]};{coord_destino[1]},{coord_destino[0]}"
        params = {
            'overview': 'full',
            'geometries': 'geojson'
        }
        
        response = requests.get(url, params=params, timeout=5)
        
        if response.status_code == 200:
            data = response.json()
            if 'routes' in data and len(data['routes']) > 0:
                # Obtener las coordenadas de la ruta
                coords = data['routes'][0]['geometry']['coordinates']
                # Convertir de [lon, lat] a [lat, lon]
                return [[c[1], c[0]] for c in coords]
    except:
        pass
    
    # Si falla, retornar línea recta
    return [[coord_origen[0], coord_origen[1]], [coord_destino[0], coord_destino[1]]]


def crear_mapa_solucion(solucion, terminales, titulo, color_rutas='green'):
    """
    Crea un mapa interactivo con Folium mostrando la red de autopistas sobre el mapa real de Buenos Aires.
    
    Parameters:
    - solucion: Grafo de NetworkX con la solución
    - terminales: Lista de ciudades principales
    - titulo: Título del mapa
    - color_rutas: Color de las autopistas en el mapa
    """
    # Centro del mapa
    centro_lat = -36.5
    centro_lon = -59.5
    
    # Crear mapa base
    mapa = folium.Map(
        location=[centro_lat, centro_lon],
        zoom_start=7,
        tiles='OpenStreetMap'
    )
    
    # Agregar título
    title_html = f'''
        <div style="position: fixed; 
                    top: 10px; left: 50px; width: 400px; height: 50px; 
                    background-color: white; border:2px solid grey; z-index:9999; 
                    font-size:16px; font-weight: bold; padding: 10px">
            {titulo}
        </div>
    '''
    mapa.get_root().html.add_child(folium.Element(title_html))
    
    # Dibujar tramos de autopista con rutas reales
    print(f"Obteniendo trazados de autopista para {solucion.number_of_edges()} tramos...")
    for i, (u, v, data) in enumerate(solucion.edges(data=True)):
        coord_u = ciudades[u]
        coord_v = ciudades[v]
        
        # Obtener ruta real
        ruta_coords = obtener_ruta_real(coord_u, coord_v)
        
        # Dibujar el tramo de autopista
        folium.PolyLine(
            locations=ruta_coords,
            color=color_rutas,
            weight=4,
            opacity=0.8,
            popup=f"<b>{u} ↔ {v}</b><br>Longitud: {data['weight']:.1f} km",
            tooltip=f"{u} → {v}"
        ).add_to(mapa)
        
        if (i + 1) % 5 == 0:
            print(f"  Procesados {i+1}/{solucion.number_of_edges()} tramos...")
    
    print("✓ Trazados completados")
    
    # Dibujar ciudades
    for nodo in solucion.nodes():
        coord = ciudades[nodo]
        
        # Determinar estilo según tipo de ciudad
        if nodo in terminales:
            color = 'red'
            icon = 'star'
            icono_color = 'white'
            tamaño = 12
            descripcion = 'Ciudad Principal (conexión obligatoria)'
        else:
            color = 'orange'
            icon = 'road'
            icono_color = 'white'
            tamaño = 10
            descripcion = 'Ciudad Intermedia (atravesada por autopista)'
        
        # Agregar marcador
        folium.Marker(
            location=[coord[0], coord[1]],
            popup=f"<b>{nodo}</b><br>{descripcion}",
            tooltip=nodo,
            icon=folium.Icon(color=color, icon=icon, prefix='fa', icon_color=icono_color)
        ).add_to(mapa)
        
        # Agregar círculo alrededor
        folium.CircleMarker(
            location=[coord[0], coord[1]],
            radius=tamaño,
            color=color,
            fill=True,
            fillColor=color,
            fillOpacity=0.3,
            weight=2
        ).add_to(mapa)
    
    return mapa


print("Funciones de mapas con trazados reales cargadas correctamente!")

Funciones de mapas con trazados reales cargadas correctamente!


In [10]:
def crear_mapa_comparativo(mst_sol, dw_sol, terminales):
    """
    Crea un mapa interactivo mostrando ambas redes de autopistas
    con capas que se pueden activar/desactivar.
    """
    # Centro del mapa
    centro_lat = -36.5
    centro_lon = -59.5
    
    # Crear mapa base
    mapa = folium.Map(
        location=[centro_lat, centro_lon],
        zoom_start=7,
        tiles='OpenStreetMap'
    )
    
    # Título
    title_html = f'''
        <div style="position: fixed; 
                    top: 10px; left: 50px; width: 500px; height: 60px; 
                    background-color: white; border:2px solid grey; z-index:9999; 
                    font-size:14px; font-weight: bold; padding: 10px">
            Comparación MST vs Dreyfus-Wagner<br>
            <span style="color:blue;">■ MST: {mst_cost:.1f} km</span> | 
            <span style="color:green;">■ Dreyfus-Wagner: {dw_cost:.1f} km</span>
        </div>
    '''
    mapa.get_root().html.add_child(folium.Element(title_html))
    
    # Crear grupos de capas
    grupo_mst = folium.FeatureGroup(name='MST (Azul)', show=True)
    grupo_dw = folium.FeatureGroup(name='Dreyfus-Wagner (Verde)', show=True)
    grupo_no_usados = folium.FeatureGroup(name='Ciudades No Utilizadas (Gris)', show=False)
    
    # Agregar tramos MST con trazados reales
    print(f"Obteniendo trazados MST ({mst_sol.number_of_edges()} tramos)...")
    for i, (u, v, data) in enumerate(mst_sol.edges(data=True)):
        coord_u = ciudades[u]
        coord_v = ciudades[v]
        
        ruta_coords = obtener_ruta_real(coord_u, coord_v)
        
        folium.PolyLine(
            locations=ruta_coords,
            color='blue',
            weight=3,
            opacity=0.6,
            popup=f"<b>MST: {u} ↔ {v}</b><br>{data['weight']:.1f} km",
            tooltip=f"MST: {data['weight']:.1f} km"
        ).add_to(grupo_mst)
        
        if (i + 1) % 3 == 0:
            print(f"  MST: {i+1}/{mst_sol.number_of_edges()} tramos...")
    
    # Agregar tramos Dreyfus-Wagner con trazados reales
    print(f"Obteniendo trazados Dreyfus-Wagner ({dw_sol.number_of_edges()} tramos)...")
    for i, (u, v, data) in enumerate(dw_sol.edges(data=True)):
        coord_u = ciudades[u]
        coord_v = ciudades[v]
        
        ruta_coords = obtener_ruta_real(coord_u, coord_v)
        
        folium.PolyLine(
            locations=ruta_coords,
            color='green',
            weight=3,
            opacity=0.6,
            popup=f"<b>D-W: {u} ↔ {v}</b><br>{data['weight']:.1f} km",
            tooltip=f"D-W: {data['weight']:.1f} km"
        ).add_to(grupo_dw)
        
        if (i + 1) % 3 == 0:
            print(f"  D-W: {i+1}/{dw_sol.number_of_edges()} tramos...")
    
    print("✓ Todos los trazados completados")
    
    # Agregar ciudades utilizadas y no utilizadas
    todas_ciudades_soluciones = set(list(mst_sol.nodes()) + list(dw_sol.nodes()))
    todas_ciudades_disponibles = set(ciudades.keys())
    ciudades_no_usadas = todas_ciudades_disponibles - todas_ciudades_soluciones
    
    # Agregar ciudades utilizadas en las soluciones
    for ciudad in todas_ciudades_soluciones:
        coord = ciudades[ciudad]
        
        if ciudad in terminales:
            color = 'red'
            icon = 'star'
            descripcion = 'Ciudad Principal (conexión obligatoria)'
            popup_text = f"<b>{ciudad}</b><br>{descripcion}"
        else:
            # Ciudad intermedia usada en alguna solución
            color = 'orange'
            icon = 'road'
            descripcion = 'Ciudad Intermedia (atravesada por autopista en D-W)'
            popup_text = f"<b>{ciudad}</b><br>{descripcion}"
        
        folium.Marker(
            location=[coord[0], coord[1]],
            popup=popup_text,
            tooltip=ciudad,
            icon=folium.Icon(color=color, icon=icon, prefix='fa', icon_color='white')
        ).add_to(mapa)
    
    # Agregar ciudades no utilizadas a la capa separada
    for ciudad in ciudades_no_usadas:
        coord = ciudades[ciudad]
        
        folium.Marker(
            location=[coord[0], coord[1]],
            popup=f"<b>{ciudad}</b><br>Ciudad Intermedia No Utilizada",
            tooltip=ciudad,
            icon=folium.Icon(color='gray', icon='circle', prefix='fa', icon_color='white')
        ).add_to(grupo_no_usados)
        
        # Agregar círculo gris alrededor
        folium.CircleMarker(
            location=[coord[0], coord[1]],
            radius=8,
            color='gray',
            fill=True,
            fillColor='gray',
            fillOpacity=0.4,
            weight=2
        ).add_to(grupo_no_usados)
    
    # Agregar capas al mapa
    grupo_mst.add_to(mapa)
    grupo_dw.add_to(mapa)
    grupo_no_usados.add_to(mapa)
    
    # Control de capas
    folium.LayerControl(collapsed=False).add_to(mapa)
    
    return mapa


print("Funciones de mapas comparativos con trazados reales cargadas!")

Funciones de mapas comparativos con trazados reales cargadas!


## Mapa Comparativo: MST vs Dreyfus-Wagner

Este mapa interactivo permite activar/desactivar las capas de MST (azul) y Dreyfus-Wagner (verde) para comparar ambas propuestas de red de autopistas.

In [11]:
# Mapa comparativo
mapa_comparativo = crear_mapa_comparativo(mst_solution, dw_solution, terminales)
mapa_comparativo

Obteniendo trazados MST (5 tramos)...
  MST: 3/5 tramos...
  MST: 3/5 tramos...
Obteniendo trazados Dreyfus-Wagner (7 tramos)...
Obteniendo trazados Dreyfus-Wagner (7 tramos)...
  D-W: 3/7 tramos...
  D-W: 3/7 tramos...
  D-W: 6/7 tramos...
  D-W: 6/7 tramos...
✓ Todos los trazados completados
✓ Todos los trazados completados


## Conclusiones

### Interpretación de Resultados:

1. **MST (Minimum Spanning Tree)**:
   - Conecta directamente solo las ciudades principales
   - Solución más simple pero requiere más kilómetros de autopista
   - No considera ciudades intermedias como puntos de paso estratégicos

2. **Dreyfus-Wagner (Steiner Tree)**:
   - Permite que la autopista pase por ciudades intermedias
   - Reduce la longitud total de infraestructura a construir
   - Solución óptima matemáticamente garantizada

3. **Aplicación Práctica en Construcción de Autopistas**:
   - Las ciudades intermedias representan puntos estratégicos de paso
   - El ahorro en kilómetros se traduce en reducción de costos de construcción
   - La solución guía decisiones sobre el trazado óptimo de la red de autopistas
   - **Importante**: A diferencia de logística, aquí solo se construye infraestructura física una vez, por lo que minimizar la longitud total es directamente proporcional al costo